# SARIMAX Model with Exogenous Variables

**Objective**: Extend ARIMA to include:
- Seasonal component (weekly cycle, m=5)
- Exogenous variables: GPR index + technical indicators

**Model**: SARIMAX(p, 0, q)(P, D, Q, 5) with lagged exog

**Critical Pattern**: ALL exogenous features MUST be lagged by 1 period to avoid future data leakage

## 1. Import Libraries

In [ ]:
# Data manipulation
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# SARIMAX modeling
from statsmodels.tsa.statespace.sarimax import SARIMAX
import pmdarima as pm

# Technical indicators
import ta

# Metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Warnings
import warnings
warnings.filterwarnings('ignore')

# Plot settings
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
plt.rcParams['figure.figsize'] = (14, 6)

print("✓ Libraries imported successfully")

## 2. Load and Prepare Data

In [ ]:
# Load dataset
df = pd.read_csv('../data/gold_silver.csv')

# Convert to datetime and set frequency
df['DATE'] = pd.to_datetime(df['DATE'])
df = df.sort_values('DATE')
df.set_index('DATE', inplace=True)
df = df.asfreq('B')  # Business Day frequency (CRITICAL for m=5)

# Calculate log returns
df['GOLD_LOG_RETURN'] = np.log(df['GOLD_PRICE']) - np.log(df['GOLD_PRICE'].shift(1))

# Filter to complete GPR data
df = df[df['GPRD'].notna()].copy()

print(f"Dataset: {len(df)} observations with complete GPR data")
print(f"Date range: {df.index.min()} to {df.index.max()}")

## 3. Feature Engineering (Technical Indicators)

In [ ]:
# Calculate technical indicators
print("Calculating technical indicators...")

# RSI (Relative Strength Index)
df['RSI'] = ta.momentum.RSIIndicator(df['GOLD_PRICE'], window=14).rsi()

# Simple Moving Average
df['SMA_20'] = df['GOLD_PRICE'].rolling(window=20).mean()

# Exponential Moving Average
df['EMA_20'] = df['GOLD_PRICE'].ewm(span=20, adjust=False).mean()

# Rolling Volatility (on log returns, NOT prices)
df['VOLATILITY_20'] = df['GOLD_LOG_RETURN'].rolling(window=20).std()

# MACD
macd = ta.trend.MACD(df['GOLD_PRICE'])
df['MACD'] = macd.macd()
df['MACD_SIGNAL'] = macd.macd_signal()

# Drop NaN from rolling calculations
df = df.dropna()

print(f"✓ Technical indicators calculated")
print(f"Remaining observations: {len(df)}")

## 4. Create Exogenous Feature Matrix (LAGGED)

**CRITICAL**: Shift all exog by 1 period to avoid future data leakage

In [ ]:
# Define exogenous features (contemporaneous)
exog_features = pd.DataFrame({
    'GPR': df['GPRD'],
    'GPR_ACT': df['GPRD_ACT'],
    'GPR_THREAT': df['GPRD_THREAT'],
    'RSI': df['RSI'],
    'SMA_20': df['SMA_20'],
    'EMA_20': df['EMA_20'],
    'VOLATILITY_20': df['VOLATILITY_20'],
    'MACD': df['MACD'],
    'MACD_SIGNAL': df['MACD_SIGNAL']
}, index=df.index)

# MANDATORY: Lag all exogenous variables by 1 period
exog_lagged = exog_features.shift(1).dropna()

# Align endogenous variable with lagged exog
endog = df['GOLD_LOG_RETURN'].loc[exog_lagged.index]

print("✓ Exogenous features created and lagged")
print(f"\nFeature matrix shape: {exog_lagged.shape}")
print(f"Endogenous variable length: {len(endog)}")
print(f"\nExog features: {exog_lagged.columns.tolist()}")

# Verify no data leakage
print(f"\n✓ Data leakage check: exog at time t contains info from t-1 (safe for forecasting)")

## 5. Train-Test Split

In [ ]:
# 80-20 split
train_size = int(len(endog) * 0.8)

train_endog = endog.iloc[:train_size]
test_endog = endog.iloc[train_size:]

train_exog = exog_lagged.iloc[:train_size]
test_exog = exog_lagged.iloc[train_size:]

# Prices for transformation
train_prices = df['GOLD_PRICE'].loc[train_endog.index]
test_prices = df['GOLD_PRICE'].loc[test_endog.index]

print(f"Train set: {len(train_endog)} observations")
print(f"Test set:  {len(test_endog)} observations")
print(f"Train dates: {train_endog.index.min()} to {train_endog.index.max()}")
print(f"Test dates:  {test_endog.index.min()} to {test_endog.index.max()}")

## 6. Model Selection with auto_arima (Seasonal)

In [ ]:
# Use auto_arima to find optimal orders
print("Running auto_arima with seasonality (m=5)...\n")
print("This may take 10-15 minutes...\n")

auto_model = pm.auto_arima(
    train_endog,
    exogenous=train_exog,
    start_p=0, max_p=3,
    start_q=0, max_q=3,
    d=0,  # Already stationary
    seasonal=True,
    m=5,  # Weekly seasonality (5 business days)
    start_P=0, max_P=2,
    start_Q=0, max_Q=2,
    D=0,  # No seasonal differencing needed
    stepwise=True,
    suppress_warnings=True,
    error_action='ignore',
    trace=True,
    information_criterion='aic',
    n_jobs=-1  # Use all CPU cores
)

print("\n" + "="*70)
print("OPTIMAL SARIMAX MODEL FOUND")
print("="*70)
print(auto_model.summary())

In [ ]:
# Extract optimal orders
best_order = auto_model.order
best_seasonal_order = auto_model.seasonal_order

print(f"\nBest SARIMAX order: {best_order}")
print(f"Best seasonal order: {best_seasonal_order}")
print(f"AIC: {auto_model.aic():.2f}")
print(f"BIC: {auto_model.bic():.2f}")

# Check if seasonality was detected
if best_seasonal_order == (0, 0, 0, 0):
    print("\n⚠ No significant seasonality detected (model reduces to ARIMAX)")
else:
    print(f"\n✓ Weekly seasonality detected with order {best_seasonal_order}")

## 7. Fit Final SARIMAX Model

In [ ]:
# Fit SARIMAX with optimal parameters
model = SARIMAX(
    train_endog,
    exog=train_exog,
    order=best_order,
    seasonal_order=best_seasonal_order,
    enforce_stationarity=False,
    enforce_invertibility=False
)

sarimax_fit = model.fit(disp=False)

print("✓ SARIMAX model fitted successfully")
print(f"\nModel: SARIMAX{best_order}{best_seasonal_order}")
print(f"Log-Likelihood: {sarimax_fit.llf:.2f}")
print(f"AIC: {sarimax_fit.aic:.2f}")
print(f"BIC: {sarimax_fit.bic:.2f}")

## 8. Coefficient Analysis

In [ ]:
# Display coefficients for exogenous variables
print("\n" + "="*70)
print("EXOGENOUS VARIABLE COEFFICIENTS")
print("="*70)

params = sarimax_fit.params
pvalues = sarimax_fit.pvalues

exog_params = pd.DataFrame({
    'Coefficient': params[params.index.str.contains('x')],
    'p-value': pvalues[pvalues.index.str.contains('x')],
    'Significant': pvalues[pvalues.index.str.contains('x')] < 0.05
})

exog_params.index = exog_lagged.columns
print(exog_params)

print(f"\n✓ Significant variables (p < 0.05): {exog_params['Significant'].sum()}/{len(exog_params)}")

## 9. Residual Diagnostics

In [ ]:
# Residuals
residuals = sarimax_fit.resid

# Plot residuals
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Residuals over time
axes[0, 0].plot(residuals, linewidth=0.5)
axes[0, 0].axhline(y=0, color='red', linestyle='--')
axes[0, 0].set_title('Residuals Over Time', fontsize=12, fontweight='bold')
axes[0, 0].set_ylabel('Residual')

# Histogram
axes[0, 1].hist(residuals, bins=50, edgecolor='black', alpha=0.7)
axes[0, 1].set_title('Residuals Distribution', fontsize=12, fontweight='bold')
axes[0, 1].set_xlabel('Residual')

# ACF
from statsmodels.graphics.tsaplots import plot_acf
plot_acf(residuals, lags=40, ax=axes[1, 0])
axes[1, 0].set_title('ACF of Residuals', fontsize=12, fontweight='bold')

# ACF of squared residuals (ARCH test)
plot_acf(residuals**2, lags=40, ax=axes[1, 1])
axes[1, 1].set_title('ACF of Squared Residuals (ARCH Effects)', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

print(f"Residuals Mean: {residuals.mean():.6f}")
print(f"Residuals Std: {residuals.std():.6f}")
print(f"\n⚠ Check squared residuals ACF for ARCH effects → if present, use GARCH (Notebook 04)")

## 10. Walk-Forward Validation (5-Day Ahead)

**Critical**: Need lagged exog values for forecast periods

In [ ]:
# Walk-forward validation
forecast_horizon = 5
predictions_log = []
actuals_log = []
predictions_price = []
actuals_price = []

print("Running walk-forward validation with SARIMAX...")
print(f"Forecast horizon: {forecast_horizon} days\n")

# Use expanding window
for i in range(0, len(test_endog) - forecast_horizon, forecast_horizon):
    # Expanding train data
    train_data_endog = pd.concat([train_endog, test_endog.iloc[:i]])
    train_data_exog = pd.concat([train_exog, test_exog.iloc[:i]])
    
    # Exog for forecast period (lagged values are already available)
    forecast_exog = test_exog.iloc[i:i+forecast_horizon]
    
    # Fit model
    model_temp = SARIMAX(
        train_data_endog,
        exog=train_data_exog,
        order=best_order,
        seasonal_order=best_seasonal_order,
        enforce_stationarity=False,
        enforce_invertibility=False
    )
    fit_temp = model_temp.fit(disp=False)
    
    # Forecast
    forecast_log = fit_temp.forecast(steps=forecast_horizon, exog=forecast_exog)
    actual_log = test_endog.iloc[i:i+forecast_horizon]
    
    predictions_log.extend(forecast_log.values)
    actuals_log.extend(actual_log.values)
    
    # Convert to prices
    last_price_idx = train_data_endog.index[-1]
    last_price = df.loc[last_price_idx, 'GOLD_PRICE']
    
    for j in range(len(forecast_log)):
        pred_price = last_price * np.exp(forecast_log.iloc[j])
        actual_price = df.loc[test_endog.index[i+j], 'GOLD_PRICE']
        predictions_price.append(pred_price)
        actuals_price.append(actual_price)
        last_price = pred_price
    
    if (i // forecast_horizon + 1) % 10 == 0:
        print(f"  Progress: {i+forecast_horizon}/{len(test_endog)} forecasts generated")

print(f"\n✓ Generated {len(predictions_price)} forecasts")

## 11. Evaluate Model Performance

In [ ]:
# Calculate metrics on PRICES
rmse = np.sqrt(mean_squared_error(actuals_price, predictions_price))
mae = mean_absolute_error(actuals_price, predictions_price)

# Load ARIMA baseline results for comparison
try:
    arima_results = pd.read_csv('../models/arima_baseline_results.csv')
    rmse_arima = arima_results['rmse'].values[0]
    mae_arima = arima_results['mae'].values[0]
    rmse_naive = arima_results['rmse_naive'].values[0]
    mae_naive = arima_results['mae_naive'].values[0]
    
    print("="*70)
    print("MODEL COMPARISON - 5-DAY AHEAD FORECASTS")
    print("="*70)
    print(f"\nSARIMAX{best_order}{best_seasonal_order} + Exog:")
    print(f"  RMSE: ${rmse:.2f}")
    print(f"  MAE:  ${mae:.2f}")
    print(f"\nARIMA Baseline (no exog):")
    print(f"  RMSE: ${rmse_arima:.2f}")
    print(f"  MAE:  ${mae_arima:.2f}")
    print(f"\nNaive Benchmark:")
    print(f"  RMSE: ${rmse_naive:.2f}")
    print(f"  MAE:  ${mae_naive:.2f}")
    print(f"\nImprovement over ARIMA:")
    print(f"  RMSE: {(1 - rmse/rmse_arima)*100:+.2f}%")
    print(f"  MAE:  {(1 - mae/mae_arima)*100:+.2f}%")
    print(f"\nImprovement over Naive:")
    print(f"  RMSE: {(1 - rmse/rmse_naive)*100:+.2f}%")
    print(f"  MAE:  {(1 - mae/mae_naive)*100:+.2f}%")
    print("="*70)
except:
    print("⚠ ARIMA baseline results not found. Run Notebook 02 first.")
    print(f"\nSARIMAX Results:")
    print(f"  RMSE: ${rmse:.2f}")
    print(f"  MAE:  ${mae:.2f}")

## 12. Visualize Forecasts

In [ ]:
# Plot predictions vs actuals
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Price forecasts
forecast_dates = test_endog.index[:len(predictions_price)]
axes[0].plot(forecast_dates, actuals_price, label='Actual Price', color='black', linewidth=1.5)
axes[0].plot(forecast_dates, predictions_price, label='SARIMAX Forecast', color='green', linewidth=1.5, alpha=0.7)
axes[0].set_title(f'SARIMAX{best_order}{best_seasonal_order} - Gold Price Forecasts (5-Day Ahead)', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Price (USD)', fontsize=11)
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Forecast errors
errors = np.array(actuals_price) - np.array(predictions_price)
axes[1].plot(forecast_dates, errors, color='red', linewidth=1)
axes[1].axhline(y=0, color='black', linestyle='--', linewidth=1)
axes[1].fill_between(forecast_dates, errors, 0, alpha=0.3, color='red')
axes[1].set_title('Forecast Errors', fontsize=14, fontweight='bold')
axes[1].set_ylabel('Error (USD)', fontsize=11)
axes[1].set_xlabel('Date', fontsize=11)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 13. Save Model Results

In [ ]:
# Save results
results = {
    'model': f'SARIMAX{best_order}{best_seasonal_order}',
    'rmse': rmse,
    'mae': mae,
    'n_predictions': len(predictions_price),
    'n_exog_features': len(exog_lagged.columns),
    'seasonal_period': 5
}

results_df = pd.DataFrame([results])
results_df.to_csv('../models/sarimax_results.csv', index=False)

# Save model
sarimax_fit.save('../models/sarimax_model.pkl')

print("✓ Model and results saved")
print("\nFiles created:")
print("  - sarimax_results.csv")
print("  - sarimax_model.pkl")

## 14. Key Findings

**Summary**:
1. SARIMAX includes GPR + technical indicators as predictors
2. All exog variables properly lagged to avoid data leakage
3. Weekly seasonality (m=5) tested
4. Compare performance vs ARIMA baseline
5. Next: Check for ARCH effects → GARCH extension (Notebook 04)